
# 🏡 Project Portfolio Lab: Predicting House Prices (Multiple Linear Regression, Ridge & LASSO)

**Dataset:** `house_prices_portfolio.csv`    
**Goal:** Build and understand linear models to predict house prices.


## 🎯 Problem Statement
A real-estate analytics team wants to estimate **house prices** in a city using property and neighborhood features. Your task is to:
1. Explore the dataset and understand the relationships.
2. Build a **Multiple Linear Regression** model and interpret results.
3. Validate **assumptions** (linearity, independence, homoscedasticity, normality, multicollinearity, outliers).
4. Apply **variable selection** and **regularization** (Ridge/LASSO) to improve generalization.
5. **Persist** the best-performing model (save & load) for future use.

> This PPL mimics the workflow you'd follow in a real project and is perfect for your portfolio.


## ✅ What you'll learn
- Load and explore a dataset (EDA)  
- Build a **Multiple Linear Regression** model (`LinearRegression`)  
- Check basic assumptions with simple plots and tests  
- Understand **overfitting vs. generalization** (train vs. test performance)  
- Improve models with **Ridge (L2)** and **LASSO (L1)** regularization  
- **Save** and **load** a trained model using `pickle`  